<a href="https://colab.research.google.com/github/TAUforPython/BioMedAI/blob/dev/NN%20LSTM%20ECG%20classification%20ArrayArrowData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт необходимых библиотек.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Bidirectional, LSTM, Dense, Masking, Dropout, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [ ]:
# -------------------------------
# Загрузка и предобработка данных
# -------------------------------

# Читаем данные из CSV файла.
data = pd.read_csv('test_data_ECG.csv')

# Убираем лишние пробелы в названиях колонок, чтобы не возникало ошибок при обращении к ним.
data = data.rename(columns=lambda x: x.strip())

# Сохраняем исходные данные в переменной clear_data для последующей работы.
clear_data = data.copy()

# Удаляем текстовые отчеты (столбцы report_0 ... report_17),
# поскольку они могут привести к переобучению модели.
reports_columns = [f'report_{x}' for x in range(18)]
clear_data = clear_data.drop(columns=reports_columns, errors='ignore')

# Преобразуем дату и время в единый формат.
# Объединяем столбцы 'eeg_date' и 'eeg_time' и конвертируем их в формат datetime.
clear_data['eeg_datetime'] = pd.to_datetime(clear_data['eeg_date'] + ' ' + clear_data['eeg_time'],
                                            errors='coerce', dayfirst=True)
# Удаляем исходные столбцы с датой и временем, так как теперь они не нужны.
clear_data = clear_data.drop(columns=['eeg_date', 'eeg_time'])

# Приводим числовые признаки к числовому типу.
# Определяем список признаков ЭКГ.
numeric_columns = ['rr_interval', 'p_onset', 'p_end', 'qrs_onset',
                   'qrs_end', 't_end', 'p_axis', 'qrs_axis', 't_axis']
clear_data[numeric_columns] = clear_data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Убираем аномальные значения: значения признаков больше 2000 считаем аномальными,
# а также удаляем строки с логически неверными соотношениями (p_onset < p_end и qrs_onset < qrs_end).
clear_data = clear_data[(clear_data[numeric_columns] < 2000).all(axis=1)]
clear_data = clear_data[(clear_data['p_onset'] < clear_data['p_end']) & (clear_data['qrs_onset'] < clear_data['qrs_end'])]

# Сортируем данные по идентификатору пациента (subject_id) и времени обследования.
# Это важно для формирования корректных временных рядов.
clear_data = clear_data.sort_values(by=['subject_id', 'eeg_datetime'])


In [ ]:
# -------------------------------
# Формирование временных рядов и нормализация
# -------------------------------

# Для каждого пациента (subject_id) формируем последовательность обследований.
# Каждая последовательность – это временной ряд, состоящий из строк с числовыми признаками ЭКГ.
# В качестве целевой метки выбирается последний зафиксированный Healthy_Status (0 — здоров, 1 — болен).
sequence_data = []
sequence_labels = []
features = numeric_columns  # Используем только числовые признаки ЭКГ.

for subject_id, group in clear_data.groupby('subject_id'):
    group_features = group[features].values  # Получаем данные в виде матрицы (seq_len, num_features)
    label = group['Healthy_Status'].values[-1]  # Берём последнюю метку пациента.
    sequence_data.append(group_features)
    sequence_labels.append(label)

# Нормализация данных:
# Объединяем все последовательности для вычисления среднего и стандартного отклонения.
all_data = np.vstack(sequence_data)
scaler = StandardScaler().fit(all_data)

# Применяем scaling к каждой последовательности индивидуально.
sequence_data_scaled = [scaler.transform(seq) for seq in sequence_data]

# Приводим все последовательности к одной длине с помощью паддинга.
# Определяем максимальную длину последовательности.
max_seq_length = max(len(seq) for seq in sequence_data_scaled)
# Используем pad_sequences для дополнения коротких последовательностей нулями (padding='post').
X = pad_sequences(sequence_data_scaled, maxlen=max_seq_length, dtype='float32', padding='post', truncating='post')
y = np.array(sequence_labels)

print("Форма входных данных:", X.shape)   # (num_patients, max_seq_length, num_features)
print("Форма меток:", y.shape)              # (num_patients,)

# Разбиваем данные на обучающую и тестовую выборки (90% на обучение, 10% на тест).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Вычисляем веса классов для компенсации дисбаланса в данных.
# Это помогает модели уделять больше внимания редкому классу.
weights = class_weight.compute_class_weight(class_weight='balanced',
                                            classes=np.unique(y_train),
                                            y=y_train)
class_weights = {i: weights[i] for i in range(len(weights))}
print("Классовые веса:", class_weights)


Форма входных данных: (6575, 4, 9)
Форма меток: (6575,)
Классовые веса: {0: np.float64(0.6990784499054821), 1: np.float64(1.755786350148368)}


In [ ]:
# -------------------------------
# Построение модели (Conv1D + Bidirectional LSTM + Глобальное объединение)
# -------------------------------

# Используем функциональный API для объединения нескольких глобальных пулов.
inputs = Input(shape=(max_seq_length, len(features)))

# Слой Masking – игнорирует паддинговые (нулевые) значения.
x = Masking(mask_value=0.)(inputs)

# Сверточный блок для выделения локальных особенностей.
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)

# Первый слой Bidirectional LSTM – извлекает временные зависимости (возвращает последовательности).
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(x)
x = BatchNormalization()(x)

# Второй слой Bidirectional LSTM – извлекает более глубокие зависимости.
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(x)
x = Dropout(0.4)(x)

# Глобальное объединение: усреднение по временной оси.
x = GlobalAveragePooling1D()(x)

# Полносвязный слой для дальнейшей обработки извлечённых признаков.
x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

# Выходной слой для бинарной классификации.
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

# Компилируем модель с дополнительными метриками.
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.AUC(name='auc')])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'conv1d_5' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 4, 9)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ masking_5 (Masking)                  │ (None, 4, 9)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 4, 64)               │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 4, 64)               │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 4, 256)              │         197,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 4, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ (None, 4, 128)              │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 4, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_4           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 373,633 (1.43 MB)

 Trainable params: 372,865 (1.42 MB)

 Non-trainable params: 768 (3.00 KB)

In [ ]:
# -------------------------------
# Обучение модели
# -------------------------------
epochs = 100
batch_size = 32

early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=epochs,
                    batch_size=batch_size,
                    class_weight=class_weights,
                    callbacks=[early_stop, reduce_lr],
                    verbose=1)


Epoch 1/100
185/185 ━━━━━━━━━━━━━━━━━━━━ 29s 62ms/step - accuracy: 0.6307 - auc: 0.7146 - loss: 0.8068 - precision: 0.4189 - recall: 0.7119 - val_accuracy: 0.7447 - val_auc: 0.7852 - val_loss: 0.6867 - val_precision: 0.5758 - val_recall: 0.3115 - learning_rate: 0.0010
Epoch 2/100
185/185 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.6876 - auc: 0.7798 - loss: 0.6551 - precision: 0.4720 - recall: 0.7875 - val_accuracy: 0.7021 - val_auc: 0.8213 - val_loss: 0.5912 - val_precision: 0.4796 - val_recall: 0.8361 - learning_rate: 0.0010
Epoch 3/100
185/185 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - accuracy: 0.7127 - auc: 0.8186 - loss: 0.5842 - precision: 0.4983 - recall: 0.8064 - val_accuracy: 0.7204 - val_auc: 0.8410 - val_loss: 0.5790 - val_precision: 0.4985 - val_recall: 0.8798 - learning_rate: 0.0010
Epoch 4/100
185/185 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.7357 - auc: 0.8365 - loss: 0.5544 - precision: 0.5346 - recall: 0.8566 - val_accuracy: 0.7340 - val_auc: 0.8427 - val_loss: 0.5

In [ ]:
# -------------------------------
# Оценка модели и оптимизация порога отсечения
# -------------------------------

# Получаем предсказания вероятностей для тестовой выборки.
y_pred_prob = model.predict(X_test).reshape(-1)

# По умолчанию порог = 0.5. Найдём оптимальный порог, максимизирующий F1-score.
best_threshold = 0.5
best_f1 = 0
for thresh in np.arange(0.3, 0.71, 0.01):
    y_pred_temp = (y_pred_prob >= thresh).astype(int)
    temp_f1 = f1_score(y_test, y_pred_temp)
    if temp_f1 > best_f1:
        best_f1 = temp_f1
        best_threshold = thresh

print(f"Оптимальный порог: {best_threshold:.2f} с F1: {best_f1:.4f}")

# Вычисляем метрики с оптимальным порогом.
y_pred = (y_pred_prob >= best_threshold).astype(int)
test_accuracy = accuracy_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test AUC-ROC: {test_auc:.4f}")


21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step
Оптимальный порог: 0.52 с F1: 0.6890
Test Accuracy: 0.7888
Test Precision: 0.5833
Test Recall: 0.8415
Test F1 Score: 0.6890
Test AUC-ROC: 0.8050


In [ ]:
# -------------------------------
# Пример предсказания для нового пациента
# -------------------------------

# Задаём данные нового пациента в виде двумерного массива (2 обследования, 9 признаков).
new_patient_data = np.array([[700, 40, 120, 160, 240, 500, 80, 75, 78],
                             [710, 40, 118, 165, 245, 510, 82, 76, 77]])  # форма: (2, num_features)

# Применяем ту же нормализацию к новым данным.
new_patient_data_scaled = scaler.transform(new_patient_data)
# Приводим последовательность нового пациента к длине, используемой моделью, с помощью паддинга.
new_patient_data_padded = pad_sequences([new_patient_data_scaled], maxlen=max_seq_length, dtype='float32',
                                         padding='post', truncating='post')
# Получаем предсказание: вероятность того, что пациент находится в критическом состоянии (1 - болен).
prediction_prob = model.predict(new_patient_data_padded)[0][0]
print("Вероятность критического состояния (1 - болен):", prediction_prob)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Вероятность критического состояния (1 - болен): 0.51075745
